In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import plotly.graph_objs as go

import plotly as py
from plotly import tools
from plotly.offline import iplot
from plotly.subplots import make_subplots
import datetime as dtt
import seaborn as sns
from fuzzywuzzy import fuzz
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dt=pd.read_csv(r'../input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv',parse_dates=['Order Date & Time'])
print("Data Dimensions are: ", dt.shape)
dt.columns=dt.columns.str.replace(" ", "_").str.lower()
print("Columns: ", dt.columns)
ct=pd.read_csv(r'../input/pakistan-cities/pk.csv')
ct.columns=ct.columns.str.lower()
#ct['short_name']=ct['area_name'].str[:3]

In [ ]:
ct.head()
ct['city'] = ct['city'].replace(['Sialkot City'],'Sialkot') 

In [ ]:
dt['order_status']=dt['order_status'].astype(str)
dt['book_name']=dt['book_name'].astype(str)
dt['city']=dt['city'].astype(str)
dt['payment_method']=dt['payment_method'].astype(str)

In [ ]:
dt['payment_method'].replace({'Cash on Delivery (COD)':'Cash on delivery'}, inplace=True)
dt['payment_method'] = dt['payment_method'].fillna('Cash on delivery')
dt['date'] = dt['order_date_&_time'].dt.date
dt['month'] = [d.month for d in dt['order_date_&_time']]
dt['month_year'] = pd.to_datetime(dt['order_date_&_time']).dt.to_period('M')
dt['time'] = dt['order_date_&_time'].dt.strftime('%H:%M')

In [ ]:
# Splliting orders having mulitple books
db = (dt.set_index(['order_number', 'order_status','order_date_&_time','city',
       'payment_method', 'total_items', 'total_weight_(grams)','date','month','month_year','time'])
    .apply(lambda x: x.str.split('/').explode())
    .reset_index())

In [ ]:
dt.drop(['order_date_&_time'],axis=1)
db=db[['order_number', 'book_name','order_status','date','month','month_year','time','city','payment_method', 'total_items', 'total_weight_(grams)']]

In [ ]:
db.city.nunique()

In [ ]:
# Applying captalize function
db.city=db.city.str.capitalize()

In [ ]:
db.city.nunique()

In [ ]:
# Replacing white spaces for easy string matching
db.city=db.city.str.replace(' ','')
db.city.nunique()


Removing duplicates that are found during exploration.

In [ ]:
db['city'].replace({'Larkans':'Larkana'}, inplace=True)
db['city'].replace({'Peshawer':'Peshawar'}, inplace=True)
db['city'].replace({'Peshwar':'Peshawar'}, inplace=True)
db['city'].replace({'Kharain':'Kharian'}, inplace=True)
db['city'].replace({'Liaquatpur':'Liaqatpur'}, inplace=True)
db['city'].replace({'Karach|کراچی|Khi|کراچی':'Karachi'}, inplace=True)
db['city'].replace({'Ahmedpureast':'Ahmadpureast'}, inplace=True)
db['city'].replace({'Dgkhan|ڈیرہغازیخان':'Deraghazikhan'}, inplace=True)
#db['city'].replace({'':'Deraismailkhan'}, inplace=True)
db['city'].replace({'ڈاکخانہخاصڈھوکمصاحبتحصیلتلہگنگضلعچکوال۔':'Chakwal'}, inplace=True)
db['city'].replace({'ملتان':'Multan'}, inplace=True)
db['city'].replace({'راولپنڈی':'Rawalpindi'}, inplace=True)
db['city'].replace({'Texila':'Taxila'}, inplace=True)
db['city'].replace({'ڈیرہاسماعیلخان':''}, inplace=True)
db['city'].replace({'ڈیرہغازیخان':'Deraghazikhan'}, inplace=True)
db['city'].replace({'جھنگ':'Jhang'}, inplace=True)
db['city'].replace({'I-9/4':'Islamabad'}, inplace=True)
db['city'].replace({'P.i.b.colony':'Karachi'}, inplace=True)
db['city'].replace({'Islamanad':'Islamabad'}, inplace=True)
db['city'].replace({'سیدوالہتحصیلوضلعننکانہصاحب':'NankanaSahib'}, inplace=True)
db['city'].replace({'گوجرانوالہ':'Gujranwala'}, inplace=True)
db['city'].replace({'جامپور':'Jampur'}, inplace=True)
db['city'].replace({'Silanwali':'Sillanwali'}, inplace=True)
db['city'].replace({'Malir':'Karachi'}, inplace=True)
db['city'].replace({'چونیاںضلعقصور':'Kasur'}, inplace=True)
db['city'].replace({'Waulahcahkwal':'Chakwal'}, inplace=True)
db['city'].replace({'Umarkot':'Umerkot'}, inplace=True)
db['city'].replace({'کوٹلہچاکرتحصیلجلالپورپیروالاملتانp/o':'Multan'}, inplace=True)

db['city'] = np.where(db['city'].str.contains(r'Mirpurak',case=False), 'Mirpur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Okaracity',case=False), 'Okara',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Rohri,districtsukkur',case=False), 'Rohri',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Dikhan|d.i.khan|ڈیرہاسماعیلخان',case=False), 'Deraismailkhan',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Qiladidarsingh,gujranwala',case=False), 'Gujranwala',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'ھریپورھزارہ|Modeltownharipur',case=False), 'Haripur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Mingoraswat',case=False), 'Mingora',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Seemafzalpurmirpurajk|Mirpurazadkashmir',case=False), 'Mirpur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Bhiriaroaddistrictn.ferozsindh|Kandiarodistrict:n-feroz',case=False), 'Naushahroferoze',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Suigasfield,suitahsil,districtdarabugti',case=False), 'Derabugti',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Lahor|Labore|Lahote|Raiwind',case=False), 'Lahore',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Noshehra|Noushehra.soonvalley',case=False), 'Nowshera',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Umarkot',case=False), 'Umerkot',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'(?!$)chakb(?!$)',case=False), 'Rawalpindi',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Fsd',case=False), 'Faisalabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Rwp',case=False), 'Rawalpindi',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'بہاولنگر',case=False), 'Bahawalnagar',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Lahoy|Lhore3|Lhr|لاہور',case=False), 'Lahore',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'خانپور',case=False), 'Khanpur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'لیاقتپورضلعرحیمیارخان',case=False), 'Rahimyarkhan',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Karchi|Gulshaneiqbal|Northnazimabad|Kornagino3|Korangi|malir|Karahi|Karach|کراچی|Khi|کراچی',case=False), 'Karachi',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'بونیر',case=False), 'Buner',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'(?!$)wah(?!$)',case=False), 'Wah',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'(?!$)ranipur(?!$)',case=False), 'Ranipur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'(?!$)khairp(?!$)|Khairpurmir',case=False), 'Khairpur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Turbatkech|Turbat,kech',case=False), 'Turbat',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Shujaabad',case=False), 'Shujabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Baatkhela',case=False), 'Batkhela',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Sadiqbad',case=False), 'Sadiqabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'اسلامآباد',case=False), 'Islamabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'لالسوہانرابہاولپور|لالسوہانرا،بہاولپور',case=False), 'Bahawalpur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Dirupper|upperdir',case=False), 'Dir',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'dirlower|lowerdir',case=False), 'Dirlower',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'(?!$)fais(?!$)',case=False), 'Faisalabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Dgkhan|D.g.khan',case=False), 'Deraghazikhan',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Rawalpindicant|Chaklalarawalpindi|Rawat',case=False), 'Rawalpindi',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Jalalpurjattangujrat',case=False), 'Gujrat',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Districtshikarpur',case=False), 'Shikarpur',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Bahriatownislamabad|Navalanchorage|Navalanchorage+923347577660|G-13/2|Ict|Isb',case=False), 'Islamabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Attockcity|Attok|Attcka',case=False), 'Attock',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'districtjacobabad',case=False), 'Jacobabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Karinwala',case=False), 'Gujrat',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Hasanabdal',case=False), 'Hassanabdal',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Shorkotcantt',case=False), 'Shorkot',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Muzafrabad|Muzzafrabad|Muzaferabad',case=False), 'Muzaffarabad',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Rawlakot',case=False), 'Rawalakot',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Quett|Queeta',case=False), 'Quetta',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Jamilchowk,ringroad',case=False), 'Peshawar',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Gwader|Gawadar',case=False), 'Gwadar',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Alipurchattha',case=False), 'Alipurchatha',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Multab',case=False), 'Multan',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Hubchoki',case=False), 'Hub',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Rawlakot',case=False), 'Rawalakot',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Muzafargarh|Muzffargarh|Distt.muzffargarh',case=False), 'Muzaffargarh',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Ahmedpureast',case=False), 'Ahmadpureast',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Timargara',case=False), 'Timergara',db['city'].str.replace(' ',''))
db['city'] = np.where(db['city'].str.contains(r'Lakkimerwat',case=False), 'Lakkimarwat',db['city'].str.replace(' ',''))


In [ ]:
db.city.nunique()

Following code will search for duplicates of top 150 cities and update the city name where match will found.

In [ ]:
cities = {}
city=db.city.value_counts()[:150].index.tolist()
for n in city:
    cities[n]= n

for n in range(len(cities)):
    key,value=list(cities.items())[n]
    db['city'] = np.where(db['city'].str.contains(value,case=False), key,db['city'].str.replace(' ',''))
    n+=1

In [ ]:
db.city.nunique()

Following function is use to found typos. It uses python fuzzywuzzy library to find match with already cleaned top 250 city names. Cities with partial match score greater than 85 will be updated.

In [ ]:
def printupdatefuzzypattrenmatch(city,matchlimit,printflag,skipcity):
    for n in city:
        q=db[db['city'].str.contains(r'(?!$)'+n[:matchlimit]+'(?!$)', case=False)].index
        #print(n)
        for i in q:
            Partial_Ratio = fuzz.partial_ratio(n.lower(),db.iloc[i].city.lower())
            if(Partial_Ratio>85 and Partial_Ratio<100):
                if(printflag == True ):
                    if n not in skipcity:
                        print(db.iloc[i].city+ '   Match against: '+n)
                else:
                    if n not in skipcity:
                        db.replace(db.iloc[i].city,n,inplace=True)
    

In [ ]:
city=db.city.value_counts()[:250].index.tolist()

In [ ]:
# Following cities will be skipped due to wrong match found during test runs
skipcity=['Kharain','Sanghar','Shahdadkot','Shahdadpur','Chishtian','Sanghar','Chakwal','Kharian','Haripur''Karak','Haripur','Chakwal','Kharian','Jhang','Hangu','Sanghar','Chishtian','Bhalwal','Town','Gojra','Tank','Kamra','Abudhabi','Alipur','Hazro','Topi','Chachransharif','Alpuri','Karak','Sanawan','Ranipur','Dina','Hala','Tandojanmuhammad']


Running fuzzypattrenmatch function after extracting first five,four,three and two characters of city names to find typting errors.

In [ ]:
printupdatefuzzypattrenmatch(city,5,True,skipcity)

In [ ]:
printupdatefuzzypattrenmatch(city,5,False,skipcity)

In [ ]:
db.city.nunique()

In [ ]:
printupdatefuzzypattrenmatch(city,4,True,skipcity)

In [ ]:
printupdatefuzzypattrenmatch(city,4,False,skipcity)

In [ ]:
printupdatefuzzypattrenmatch(city,3,True,skipcity)

In [ ]:
printupdatefuzzypattrenmatch(city,3,False,skipcity)

In [ ]:
db.city.nunique()

In [ ]:
printupdatefuzzypattrenmatch(city,2,True,skipcity)

In [ ]:
printupdatefuzzypattrenmatch(city,2,False,skipcity)

Almost 70% cleansing of the city names have been achieved. 

In [ ]:
db.city.nunique()

Following graphs have been taken from https://www.kaggle.com/alikhan83/story-of-gufhtugu-publications/ notebook.

In [ ]:
geo=db['city'].value_counts().rename_axis('City').reset_index(name='counts')
geo.head()
clean_geo=geo.merge(ct,how='inner',left_on='City', right_on='city')
clean_geo=clean_geo[["City","lat","lng","admin_name",
         "population_proper","counts"]]
clean_geo=clean_geo.rename(columns={"lat":"Latitude","lng":"Longitude",
                                    "population_proper":"Population",
                                    "admin_name":"Province","counts":"Total Orders"})
clean_geo.head()

In [ ]:
import plotly.graph_objects as go


clean_geo['text'] = clean_geo['City'] + '<br> Book Sold ' + (clean_geo['Total Orders']).astype(str)
#limits = [(0,5),(5,20),(20,50),(50,500),(500,1250)]
limits = [(0,3),(3,9),(9,19),(19,49),(49,1300)] 

colors = ["royalblue","crimson","lightseagreen","orange","lightgrey"]
scale = 7000

fig = go.Figure()

for i in range(len(limits)):
    lim = limits[i]
    df_sub = clean_geo[lim[0]:lim[1]]
    fig.add_trace(go.Scattergeo(
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['Total Orders'],
            color = colors[i],
            line_color='rgb(40,40,40)',
            line_width=0.5,
            sizemode = 'area'
        ),
        name = 'Top {0} - {1}'.format(lim[0]+1,lim[1])))

fig.update_layout(
        title_text = 'Total Books Sold by Gufhtugu <br>(Click legend to toggle traces)',
        showlegend = True,
        legend_title="Top Books",
        legend_title_font_size=14,
        geo = dict(
            scope = 'asia',
            landcolor = 'rgb(217, 217, 217)',
            lonaxis = dict(range = [60.578993, 82.65129]),
            lataxis = dict(range = [24.407138, 36.885931]),
            
        ),
        
    )

fig.show()

In [ ]:
month=db[['city','month']].value_counts().rename_axis(['City','Month']).reset_index(name='counts')
month_geo=month.merge(clean_geo,how='inner',left_on='City', right_on='City').sort_values(['Month'])
month_geo.head()

In [ ]:
import plotly.express as px
px.scatter(month_geo, x="Total Orders", y="counts", animation_frame="Month", animation_group="City",
           size="counts", color="Province", hover_name="City",title='Number of Books bought by cities over month',
            log_x=True, size_max=40, range_x=[1,8000], range_y=[0,1500])